# Scripts just to test YOLO models

## Train on random weights

In [ ]:
from ultralytics import YOLO
model = YOLO("yolo12n.pt")

results = model.train(data="coco8.yaml", epochs=5, imgsz=640, device='cuda')

## Train on custom dataset

### Yolo

In [ ]:
from ultralytics import YOLO
model = YOLO("yolo12n.pt")

results = model.train(data="/home/lucas-fontoura/Documents/yolo-drone/data/datasets/veryreduced-sliced-sahi-yolodrone/data.yaml", epochs=10, imgsz=640, device='cuda', batch=16)

In [ ]:
metrics = model.val()

In [ ]:
results = {
    "map50-95 (B)": metrics.box.map,
    "map50 (B)": metrics.box.map50,
    "map75 (B)": metrics.box.map75,
    "map50-95 (B) of each category": metrics.box.maps,
}

results

In [ ]:
# Predict with the model
results = model("/home/lucas-fontoura/Documents/yolo-drone/data/datasets/VeryReduced_InfectedLeaves/images/Aug03_USAMV_DJI_0562_f_01100.jpg", save=True)  # predict on an image

# Access the results
for result in results:
    xy = result.masks.xy  # mask in polygon format
    xyn = result.masks.xyn  # normalized
    masks = result.masks.data  # mask in matrix format (num_objects x H x W)

### Yolo-seg

In [ ]:
from ultralytics import YOLO
model = YOLO("yolo11n-seg.pt")

results = model.train(data="/home/lucas-fontoura/Documents/yolo-drone/data/datasets/synthetic-seg/data.yaml", epochs=50, imgsz=800, device='cuda', batch=16)

In [ ]:
metrics = model.val()

In [ ]:
results = {
    "map50-95 (B)": metrics.seg.map,
    "map50 (B)": metrics.seg.map50,
    "map75 (B)": metrics.seg.map75,
    "map50-95 (B) of each category": metrics.seg.maps,
}

results

In [ ]:
from ultralytics import YOLO
# Predict with the model
model = YOLO("/home/lucas-fontoura/Documents/yolo-drone/runs/segment/train3/weights/best.pt")

results = model("/home/lucas-fontoura/Documents/yolo-drone/data/datasets/synthetic-seg/images/train/", save=False, save_conf=True, save_txt=True)  # predict on an image

In [ ]:
import os
import csv
from ultralytics import YOLO
import cv2
import numpy as np

# ----------------------------
# SETTINGS
# ----------------------------
MODEL_PATH = "/home/lucas-fontoura/Documents/yolo-drone/runs/segment/train3/weights/best.pt"
DATASET_PATH = "/home/lucas-fontoura/Documents/yolo-drone/data/datasets/synthetic-seg/images/train/"
LABELS_PATH = "/home/lucas-fontoura/Documents/yolo-drone/data/datasets/synthetic-seg/labels/train/"  # YOLO txt labels for test set
OUTPUT_DIR = "difficult_cases"
CONF_THRESHOLD = 0.25   # prediction confidence threshold
HIGH_CONF_THRESHOLD = 0.8  # "confident but wrong"
IOU_THRESHOLD = 0.2     # overlap threshold for matching preds <-> GT
SAVE_VISUALS = True

# ----------------------------
# HELPER FUNCTIONS
# ----------------------------
def box_iou(box1, box2):
    """Compute IoU between two boxes [x1,y1,x2,y2]."""
    xA = max(box1[0], box2[0])
    yA = max(box1[1], box2[1])
    xB = min(box1[2], box2[2])
    yB = min(box1[3], box2[3])
    inter = max(0, xB - xA) * max(0, yB - yA)
    area1 = (box1[2]-box1[0]) * (box1[3]-box1[1])
    area2 = (box2[2]-box2[0]) * (box2[3]-box2[1])
    union = area1 + area2 - inter
    return inter / union if union > 0 else 0

def load_gt_boxes(label_path, img_w, img_h):
    """Load YOLO-Seg txt labels (class + polygon points) and convert to bounding boxes in pixels."""
    if not os.path.exists(label_path):
        return []
    boxes = []
    with open(label_path, "r") as f:
        for line in f:
            parts = line.strip().split()
            cls = int(parts[0])
            coords = list(map(float, parts[1:]))
            xs = coords[0::2]
            ys = coords[1::2]
            # denormalize
            xs = [int(x * img_w) for x in xs]
            ys = [int(y * img_h) for y in ys]
            x1, y1, x2, y2 = min(xs), min(ys), max(xs), max(ys)
            boxes.append([x1, y1, x2, y2, cls])
    return boxes

def draw_boxes(image, preds, gts):
    """Overlay predictions (green) and GT (red)."""
    for (x1,y1,x2,y2,cls,conf) in preds:
        cv2.rectangle(image, (x1,y1), (x2,y2), (0,255,0), 2)
        cv2.putText(image, f"P:{cls} {conf:.2f}", (x1,y1-5), 
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,255,0), 1)
    for (x1,y1,x2,y2,cls) in gts:
        cv2.rectangle(image, (x1,y1), (x2,y2), (0,0,255), 2)
        cv2.putText(image, f"GT:{cls}", (x1,y2+15), 
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,255), 1)
    return image

# ----------------------------
# MAIN
# ----------------------------
os.makedirs(OUTPUT_DIR, exist_ok=True)
model = YOLO(MODEL_PATH)

results = model(DATASET_PATH, conf=CONF_THRESHOLD, save=False)

report = []

for r in results:
    img_path = r.path
    img = cv2.imread(img_path)
    h, w = img.shape[:2]
    base = os.path.splitext(os.path.basename(img_path))[0]

    # Ground truth
    label_path = os.path.join(LABELS_PATH, base + ".txt")
    gts = load_gt_boxes(label_path, w, h)

    # Predictions
    preds = []
    for b in r.boxes.data.cpu().numpy():
        x1, y1, x2, y2, conf, cls = b
        preds.append([int(x1), int(y1), int(x2), int(y2), int(cls), float(conf)])

    # Count errors
    high_conf_fp = [p for p in preds if p[5] > HIGH_CONF_THRESHOLD and 
                    max([box_iou(p, g) for g in gts] or [0]) < IOU_THRESHOLD]

    missed_gt = [g for g in gts if max([box_iou(p, g) for p in preds] or [0]) < IOU_THRESHOLD]

    difficulty_score = len(high_conf_fp) + len(missed_gt)

    report.append([img_path, difficulty_score, len(high_conf_fp), len(missed_gt)])

    # Save visuals
    if SAVE_VISUALS and difficulty_score > 0:
        vis_img = draw_boxes(img.copy(), preds, gts)
        cv2.imwrite(os.path.join(OUTPUT_DIR, base + "_vis.jpg"), vis_img)

# Save CSV report
report.sort(key=lambda x: x[1], reverse=True)
with open(os.path.join(OUTPUT_DIR, "difficult_images.csv"), "w", newline="") as f:
    writer = csv.writer(f)
    writer.writerow(["image", "difficulty_score", "high_conf_fp", "missed_gt"])
    writer.writerows(report)

print(f"Analysis complete. Report saved to {OUTPUT_DIR}/difficult_images.csv")
if SAVE_VISUALS:
    print(f"Visual samples saved to {OUTPUT_DIR}/")


In [ ]:
from ultralytics import YOLO

model = YOLO("yolo12n.pt")

In [ ]:
old_conv = model.model.model[0].conv

In [ ]:
old_conv